In [0]:
data = [line.strip().split('\t') for line in open('project3_dataset1.txt', 'r')]

In [0]:
import numpy as np
data = np.asarray(data)
print(data)
classlabels = data[:,-1].reshape((data.shape[0],1)).astype(int)
data = data[:,0:-1]
print(classlabels)
print(data)

[['20.13' '28.25' '131.2' ... '0.2572' '0.06637' '1']
 ['12.45' '15.7' '82.57' ... '0.3985' '0.1244' '1']
 ['11.26' '19.96' '73.72' ... '0.2955' '0.07009' '0']
 ...
 ['14.5' '10.89' '94.28' ... '0.2889' '0.08006' '0']
 ['12.36' '18.54' '79.01' ... '0.2983' '0.07185' '0']
 ['11.93' '21.53' '76.53' ... '0.2438' '0.08541' '0']]
[[1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0

In [0]:
def is_categorical(s):
    try:
        float(s)
    except ValueError:
        return True
    return False

In [0]:
strindex = []
for i in range(len(data[0])):
    if(is_categorical(data[0][i])):
        print(data[0][i])
        print(i)
        strindex.append(i)
for i in strindex:
    unique_vals = np.unique(data[:,i])
    mapping = list(range(len(unique_vals)))
    dictionary = dict(zip(unique_vals, mapping))
    for j in range(len(data[:,i])):
        data[j][i] = dictionary.get(data[j][i])

data = data.astype(float)
data = np.append(data,classlabels,axis=1)
print(data)

[[2.013e+01 2.825e+01 1.312e+02 ... 2.572e-01 6.637e-02 1.000e+00]
 [1.245e+01 1.570e+01 8.257e+01 ... 3.985e-01 1.244e-01 1.000e+00]
 [1.126e+01 1.996e+01 7.372e+01 ... 2.955e-01 7.009e-02 0.000e+00]
 ...
 [1.450e+01 1.089e+01 9.428e+01 ... 2.889e-01 8.006e-02 0.000e+00]
 [1.236e+01 1.854e+01 7.901e+01 ... 2.983e-01 7.185e-02 0.000e+00]
 [1.193e+01 2.153e+01 7.653e+01 ... 2.438e-01 8.541e-02 0.000e+00]]


In [0]:
# print(data[:,4])
print(data.shape)

(569, 31)


In [0]:
def split(kfoldlist,index):
    test = kfoldlist[index]
    train = np.vstack([x for i,x in enumerate(kfoldlist) if i != index])
    train = np.asarray(train)
    test = np.asarray(test)
    return train, test

def normalize(train_data, test_point):
    tempdata = np.vstack((train_data[:,0:-1],test_point[0:-1].reshape((1,test_point[0:-1].shape[0]))))
#     print("tempdata: ",tempdata.shape)
    datanew = (tempdata - np.mean(tempdata,axis = 0))/np.std(tempdata,axis = 0)
    train_new = np.hstack((datanew[0:-1,:],train_data[:,-1].reshape(train_data[:,-1].shape[0],1)))
#     print("train_new: ",train_new.shape)
    test_new = np.hstack((datanew[-1,:],test_point[-1]))
#     print("test_point[-1]: ",test_point[-1])
#     print("test_new: ",test_new)
    return train_new, test_new

In [0]:
def calculate_dist(p1,p2):
    sum = 0
    for i in range(len(p2)):
        if(i in strindex):
            if(p2[i] != p1[i]):
                sum += 1 
            else:
                sum += 0
        else:
            sum += ((p1[i] - p2[i]) ** 2)
    dist = sum**0.5        
    return dist


def classify(train_data, point_to_classify):
    dist_list = []
    for tr in train_data:
        dist_list.append(calculate_dist(tr[0:-1],point_to_classify[0:-1]))
    closestk = np.argsort(np.asarray(dist_list))[:k]
    zeros = 0
    ones = 0
#     print(len(closestk))
    for c in closestk:
#         print(c)
        if(train_data[:,-1][c] == 0):
            zeros +=1
        elif(train_data[:,-1][c] == 1):
            ones +=1
#     print("------------------------------------------------------------------------")        
    return 1 if(ones>zeros) else 0
    
    

In [0]:
def evaluate(actual, predicted):
    # a =TP, b=FN, c= FP, d= TN
    a = b = c = d = 0
    for i in range(len(actual)):
        if(actual[i] == 1 and predicted[i] == 1):
            a += 1
        elif(actual[i] == 1 and predicted[i] == 0):
            b += 1
        elif(actual[i] == 0 and predicted[i] == 1):
            c += 1
        elif(actual[i] == 0 and predicted[i] == 0):
            d += 1
    return a, b, c, d

In [0]:
kfoldlist = np.array_split(data, 10)
accuracy = precision = recall = f_measure = 0
k=9
for i in range(len(kfoldlist)):
    train, test = split(kfoldlist,i)
    test_labels = []
    for j in range(len(test)):
        train_new, test_new = normalize(train, test[j])
        test_labels.append(classify(train_new, test_new))
    print(len(test_labels)) 
    test_labels = np.asarray(test_labels)
    
    a, b, c, d = evaluate(test[:,-1], test_labels)
    accuracy += (float(a + d)/(a + b + c + d))
    precision += (float(a)/(a + c))
    recall += (float(a)/(a + b))
    f_measure += (float(2 * a) / ((2 * a) + b + c))
    
print("Accuracy: ",accuracy*10,"%")
print("Precision: ",precision*10,"%")
print("Recall: ",recall*10,"%")
print("F-1 Measure: ",f_measure*10,"%")   

57
57
57
57
57
57
57
57
57
56
Accuracy:  96.66353383458647 %
Precision:  98.54219948849105 %
Recall:  92.58877591865794 %
F-1 Measure:  95.29281749376221 %
